In [12]:
import sys
import eastern
import pytz
import numpy as np
import pandas as pd
import shinybroker as sb
from datetime import timedelta

In [2]:
asset_symbol = "MSFT"

In [3]:
############ Risk Tolerance Calculation
asset_data_vix = sb.fetch_historical_data(
    contract=sb.Contract({
        'symbol': 'VIX',  # or 'VX' for VIX futures
        'secType': 'IND',  # Futures contract type
        'exchange': 'CBOE',  # CBOE Futures Exchange for VIX futures
        'currency': 'USD',
    }),
    barSizeSetting='1 day',
    durationStr='1 Y'
)
print(asset_data_vix)
# Extract the DataFrame from the result
df = asset_data_vix['hst_dta']

# Ensure timestamp is in datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Create a new DataFrame with just the date and scaled high value
risktol_df = pd.DataFrame({
    'Timestamp': df['timestamp'].dt.date,
    'risk_tolerance': df['high'] * 0.01
})

# Print header
print("Timestamp   risk_tolerance")

# Print each row
for _, row in risktol_df.iterrows():
    print(f"{row['Timestamp']}  {row['risk_tolerance']:.4f}")


{'startDateStr': '20240501 18:50:40 US/Central', 'endDateStr': '20250501 18:50:40 US/Central', 'hst_dta':       timestamp   open   high    low  close  volume  wap  barCount
0    2024-05-02  15.14  16.09  14.60  14.68       0  0.0      1407
1    2024-05-03  14.51  14.58  13.48  13.49       0  0.0      1187
2    2024-05-06  13.98  14.02  13.44  13.49       0  0.0       698
3    2024-05-07  13.52  13.64  13.16  13.23       0  0.0       734
4    2024-05-08  13.24  13.51  12.94  13.00       0  0.0       854
..          ...    ...    ...    ...    ...     ...  ...       ...
245  2025-04-25  26.22  27.20  24.84  24.84       0  0.0      2279
246  2025-04-28  25.75  26.93  24.70  25.15       0  0.0      2092
247  2025-04-29  24.76  25.99  23.76  24.17       0  0.0      2246
248  2025-04-30  24.35  28.17  24.23  24.70       0  0.0      2225
249  2025-05-01  23.94  25.18  23.30  24.60       0  0.0      2157

[250 rows x 8 columns]}
Timestamp   risk_tolerance
2024-05-02  0.1609
2024-05-03  0.1458


In [4]:
############ Asset related
asset_ask_data = sb.fetch_historical_data(
    contract=sb.Contract({
        'symbol': asset_symbol,
        'secType': 'STK',
        'exchange': 'SMART',
        'currency': 'USD',
    }),
    barSizeSetting='30 secs',
    durationStr='1 D',
    whatToShow = 'ASK'
)
asset_bid_data = sb.fetch_historical_data(
    contract=sb.Contract({
        'symbol': asset_symbol,
        'secType': 'STK',
        'exchange': 'SMART',
        'currency': 'USD',
    }),
    barSizeSetting='30 secs',
    durationStr='1 D',
    whatToShow = 'BID'
)

ask_df = asset_ask_data['hst_dta'][['timestamp', 'low']].rename(columns={'low': 'ask_low'})
bid_df = asset_bid_data['hst_dta'][['timestamp', 'high']].rename(columns={'high': 'bid_high'})

merged_df = ask_df.merge(bid_df, on='timestamp')
merged_df['mid_price'] = (merged_df['ask_low'] + merged_df['bid_high']) / 2
pd.set_option('display.max_rows', None)
#print(merged_df)

############ Set initial parameters
#Rolling Std of Mid-Price Returns
merged_df['log_return'] = np.log(merged_df['mid_price'] / merged_df['mid_price'].shift(1))
window = 30  # Example: 60 periods
merged_df['rolling_volatility'] = merged_df['log_return'].rolling(window=window).std()
merged_df.dropna(subset=['rolling_volatility'], inplace=True)
pd.set_option('display.max_rows', None)
print(merged_df)

              timestamp  ask_low  bid_high  mid_price    log_return  \
30  2025-05-01 09:45:00   434.19    434.59    434.390  0.000000e+00   
31  2025-05-01 09:45:30   434.09    434.66    434.375 -3.453178e-05   
32  2025-05-01 09:46:00   433.89    434.28    434.085 -6.678489e-04   
33  2025-05-01 09:46:30   433.69    434.37    434.030 -1.267113e-04   
34  2025-05-01 09:47:00   433.73    434.06    433.895 -3.110868e-04   
35  2025-05-01 09:47:30   434.04    434.32    434.180  6.566253e-04   
36  2025-05-01 09:48:00   433.62    434.31    433.965 -4.953090e-04   
37  2025-05-01 09:48:30   433.73    434.39    434.060  2.188877e-04   
38  2025-05-01 09:49:00   433.88    434.52    434.200  3.224841e-04   
39  2025-05-01 09:49:30   433.26    433.79    433.525 -1.555793e-03   
40  2025-05-01 09:50:00   432.41    433.32    432.865 -1.523564e-03   
41  2025-05-01 09:50:30   432.29    432.85    432.570 -6.817381e-04   
42  2025-05-01 09:51:00   432.55    433.21    432.880  7.163903e-04   
43  20

In [7]:
import pandas as pd
import numpy as np

# ===== TUNABLE PARAMETERS =====
gamma = 10
tolerance_bps = 150
initial_inventory = -1
volatility_multiplier = 20
min_time_diff = 1e-6
# ==============================

df = merged_df.copy()
inventory = initial_inventory
cash_position = 1_000_000.0  # Starting with $1,000,000
reservation_prices = []
optimal_spreads = []
blotter = []
ledger = []

# Precompute constant term in optimal spread
constant_spread_term = (2 / gamma) * np.log(1 + 1 / gamma)
last_fill_time = pd.to_datetime(df.iloc[0]['timestamp'])

# === Main Loop ===
for i, row in df.iterrows():
    current_time = pd.to_datetime(row['timestamp'])
    st = row['mid_price']
    ask = row['ask_low']
    bid = row['bid_high']

    variance = (row['rolling_volatility'] * volatility_multiplier) ** 2
    time_diff = (current_time - last_fill_time).total_seconds() / 60
    time_in_market = max(time_diff, min_time_diff)

    rt = st - gamma * inventory * variance * time_in_market
    reservation_prices.append(rt)

    spread = gamma * variance * time_in_market + constant_spread_term
    optimal_spreads.append(spread)

    tolerance = (tolerance_bps / 10000) * st

    trade = None
    fill_price = None

    if rt < ask and abs(rt - ask) <= tolerance:
        trade = 'BUY'
        fill_price = ask
        inventory += 1
        cash_position -= fill_price
        last_fill_time = current_time

    elif rt > bid and abs(rt - bid) <= tolerance:
        trade = 'SELL'
        fill_price = bid
        inventory -= 1
        cash_position += fill_price
        last_fill_time = current_time

    # Inventory rebalancing logic
    if inventory == 10:
        inventory -= 11
        fill_price = bid  # use bid for sell
        cash_position += bid * 11
        trade = 'SELL'
        last_fill_time = current_time

    # Portfolio value = cash + inventory × mid price
    portfolio_value = cash_position + inventory * st
    unrealized_pnl = inventory * (st - fill_price) if fill_price is not None else 0
    total_pnl = portfolio_value  # cash + inventory * mid_price

    # Record trade to blotter
    if trade and fill_price is not None:
        blotter.append({
            'timestamp': current_time,
            'trade': trade,
            'fill_price': round(fill_price, 2),
            'reservation_price': round(rt, 2),
            'inventory': inventory,
            'cash_position': round(cash_position, 3),
            'portfolio_value': round(portfolio_value, 3)
        })

    # Record every step in the ledger (regardless of trade)
    ledger.append({
        'timestamp': current_time,
        'inventory': inventory,
        'cash_position': round(cash_position, 3),
        'mid_price': round(st, 2),
        'portfolio_value': round(portfolio_value, 3),
        'unrealized_pnl': round(unrealized_pnl, 3)
    })

# Assign to main df
df['reservation_price'] = reservation_prices
df['optimal_spread'] = optimal_spreads

# Final outputs
blotter_df = pd.DataFrame(blotter)
ledger_df = pd.DataFrame(ledger)

# Display blotter and ledger
print("=== Blotter ===")
print(blotter_df[['timestamp', 'trade', 'fill_price', 'inventory', 'cash_position']])

print("\n=== Ledger ===")
print(ledger_df[['timestamp', 'inventory', 'cash_position', 'mid_price', 'portfolio_value']])

#Save ledger to CSV
ledger_df.to_csv("ledger.csv", index=False)


=== Blotter ===
              timestamp trade  fill_price  inventory  cash_position
0   2025-04-21 09:54:30   BUY      362.30          0      999637.70
1   2025-04-21 10:35:30   BUY      360.50          1      999277.20
2   2025-04-21 10:43:00   BUY      361.07          2      998916.13
3   2025-04-21 10:47:30   BUY      360.44          3      998555.69
4   2025-04-21 10:48:00   BUY      360.48          4      998195.21
5   2025-04-21 10:52:30   BUY      359.98          5      997835.23
6   2025-04-21 10:58:00   BUY      359.98          6      997475.25
7   2025-04-21 11:00:30   BUY      360.34          7      997114.91
8   2025-04-21 11:02:30   BUY      359.97          8      996754.94
9   2025-04-21 11:07:30   BUY      360.60          9      996394.34
10  2025-04-21 11:08:00  SELL      360.58         -1     1000000.10
11  2025-04-21 11:09:30  SELL      360.90         -2     1000361.00
12  2025-04-21 11:12:30  SELL      360.82         -3     1000721.82
13  2025-04-21 11:21:00   BUY   